In [ ]:
!pip install selenium

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import time

In [2]:
browser = webdriver.Chrome('C:\chromedriver.exe')
browser.get('https://www.bigkinds.or.kr/v2/news/index.do')

In [5]:
# 기간 1주일로 설정

browser.find_element_by_xpath('//*[@id="collapse-step-1-body"]/div[3]/div/div[1]/div[1]/a').click()
time.sleep(1)
browser.find_element_by_xpath('//*[@id="srch-tab1"]/div/div[1]/span[2]').click()

In [6]:
# #검색창의 이름을 찾아서 사용자가 검색어 입력

query_txt = input('크롤링할 키워드는 무엇입니까?:')
element = browser.find_element_by_id("total-search-key")
element.send_keys(query_txt)  
element.send_keys("\n")  #엔터효과

크롤링할 키워드는 무엇입니까?:넥슨


In [7]:
# 경제 분류, 기사 100건 씩 보기 선택

from selenium.webdriver.common.keys import Keys
browser.find_element_by_xpath('//*[@id="filterTab03"]/li[2]/span').send_keys(Keys.ENTER)
time.sleep(1)
browser.find_element_by_xpath('//*[@id="select2"]/option[4]').send_keys(Keys.ENTER)
# .click()

In [8]:
html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')

//*[@id="news-results-tab"]/div[6]/div[1]/div/div/div/div[4]/a

#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(7) > a

In [14]:
# 다음 페이지 넘기기

nextbtn = browser.find_element_by_css_selector('#news-results-tab > div.data-result-btm.m-only.paging-v3-wrp > div.btm-pav-wrp > div > div > div > div:nth-child(7) > a')
nextbtn.send_keys(Keys.ENTER)

In [15]:
wholes = soup.select('div.news-inner')
len(wholes)

40

In [ ]:
# [제목, 언론사, 분류, 날짜, 기자이름, 기사내용]

contents = [] 
for whole in wholes:
    title = whole.select('span.title-elipsis')
    press =  whole.select('div.info > div > a')
    category = whole.select('div.info > div > span.bullet-keyword')
    date = whole.select('div.info > p')
    contents.append([title[0].text.strip(), press[0].text.strip(), category[0].text.strip(), date[0].text.strip(), date[1].text.strip()])

len(contents)

In [ ]:
contents[1]

In [ ]:
# 기사 전문 

# texts = []
# for i in range(0, 100):
#     browser.find_elements_by_css_selector('span.title-elipsis')[i].click()
#     time.sleep(1)
#     text = browser.find_elements_by_css_selector('div.news-view-body')[0].text
#     texts.append(text.replace('\n', ''))
#     browser.find_element_by_xpath("//div[@id='news-detail-modal']/div/div/button").click()

# len(texts)

In [ ]:
df = pd.DataFrame(data=contents, columns =['title', 'press', 'category', 'date', 'name'])